In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('AASG_Thermed_AllTempsThicksConds.csv', low_memory=False)

In [3]:
def remove_outliers_iqr(data, column):
    # Calculate the first and third quartiles
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    
    # Calculating the IQR (Interquartile Range)
    IQR = Q3 - Q1
    
    # Defining the lower and upper bounds to identify outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Creating masks for outliers and non-outliers
    outliers_mask = (data[column] < lower_bound) | (data[column] > upper_bound)
    # Creating a mask for non-outliers
    non_outliers_mask = (data[column] >= lower_bound) & (data[column] <= upper_bound)
    
    return data[non_outliers_mask].copy(), outliers_mask

In [4]:
df = remove_outliers_iqr(df, 'HeatFlow')[0]

In [5]:
df = df.reset_index(drop=True)

In [6]:
# Forming X and Y
X = np.transpose(np.array([df.LatDegree,
                                df.LongDegree,
                                df.MeasureDepth_m,
                                df.SurfTemp]))
Y = df.CorrBHT.values
# Adding Geological Layer information to X
layers = df.iloc[:,52:101].values
conds = df.iloc[:,101:150].values
mult = np.multiply(layers,conds)
np.nan_to_num(mult, 0)
X = np.concatenate((X, mult),axis=1)

In [7]:
df2 = pd.read_csv('clean_new_well_data_fixed.csv')
num_sample=10000
sampled_df2 = df2.sample(num_sample)
sampled_df2.reset_index(inplace=True,drop=True)
sampled_df2.head()

,id,depth,temp,lat,lon,corrtemp
0,4705300071,1221.1050,39.500000,38.535339,-82.065496,45.471605
1,4702105522,914.0952,26.727778,38.931402,-80.928307,30.697678
2,4704500496,1453.2864,29.937389,37.724143,-81.860383,37.422816
3,4710300645,1553.4894,43.600000,39.678222,-80.823766,51.738751
4,4704500496,1516.0752,31.854500,37.724143,-81.860383,39.749310


In [8]:
# Interpolating Geological Layer information for X
lat_to_interpolate = sampled_df2.lat'
lon_to_interpolate = sampled_df2.lon
layers = df.iloc[:,52:101].values
conds = df.iloc[:,101:150].values
mult = np.multiply(layers,conds)
np.nan_to_num(mult, 0)

f = open("optim_result.out", "r")
lines = f.readlines()

optimal_neigh = []
optimal_width = []
for line in lines:
    optimal_neigh.append(line.split(',')[0][0])
    optimal_width.append(line.split(',')[1])
optimal_neigh = np.array(optimal_neigh).astype('int')
optimal_width = np.array(optimal_width).astype('float')

# Predicting 49 layers information for each sampled_df2 lat and lon
from sklearn.neighbors import KNeighborsRegressor
predicted_mults = []
for i in range(0,49):
    def gaussian_kernel(distances):
                kernel_width = optimal_width[i]
                weights = np.exp(-(distances**2)/kernel_width)
                return weights
    knn = KNeighborsRegressor(n_neighbors=optimal_neigh[i],weights=gaussian_kernel)
    #knn = KNeighborsRegressor(n_neighbors=1,weights=gaussian_kernel)
    knn.fit(np.transpose(np.array([df.LatDegree, df.LongDegree])), mult[:,i])
    y_pred = knn.predict(np.transpose(np.array([sampled_df2.lat, sampled_df2.lon])))
    predicted_mults.append(y_pred)
    
predicted_mults = np.transpose(np.array(predicted_mults))

# Predicting T_SURF
def gaussian_kernel(distances):
            kernel_width = 2.598
            weights = np.exp(-(distances**2)/kernel_width)
            return weights
knn = KNeighborsRegressor(n_neighbors=1,weights=gaussian_kernel)
knn.fit(np.transpose(np.array([df.LatDegree, df.LongDegree])), df.SurfTemp)
predicted_tsurf = knn.predict(np.transpose(np.array([sampled_df2.lat, sampled_df2.lon])))

In [9]:
# Forming X and Y
new_X = np.transpose(np.array([sampled_df2.lat,
                                sampled_df2.lon,
                                sampled_df2.depth,
                                predicted_tsurf]))
new_X = np.concatenate((new_X, predicted_mults),axis=1)
new_Y = sampled_df2.corrtemp.values

# Random Forest for New Well Data

In [13]:
import sklearn.metrics as m
from sklearn.ensemble import RandomForestRegressor

In [14]:
def Rftest():
    model = RandomForestRegressor(max_depth=10, n_estimators=50)
    model.fit(X, Y)
    y_pred = model.predict(new_X)
    y_test = new_Y
    std = (np.nanstd(abs(y_test-y_pred)))
    return m.mean_absolute_error(y_test, y_pred), m.mean_squared_error(y_test, y_pred), std

In [15]:
mae, mse, std = Rftest()
print(mae, mse, std) 

7.774905613788625 97.49573865939425 6.086590289815176


# LightGBM for New Well Data

In [16]:
import lightgbm as lgb

In [17]:
def LightGBM():
    model = lgb.LGBMRegressor(learning_rate=0.1, num_leaves=30, n_estimators=200,
                              max_depth=15, min_child_samples=5)
    model.fit(X, Y)
    y_pred = model.predict(new_X)
    y_test = new_Y
    std = (np.nanstd(abs(y_test-y_pred)))
    return m.mean_absolute_error(y_test, y_pred), m.mean_squared_error(y_test, y_pred), std

In [18]:
mae, mse, std = LightGBM()
print(mae, mse, std) 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013612 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13062
[LightGBM] [Info] Number of data points in the train set: 19593, number of used features: 53
[LightGBM] [Info] Start training from score 37.071158
7.481264314310741 92.42281300032357 6.037673166025483


# DNN for New Well Data

In [44]:
from keras import models
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from sklearn.model_selection import train_test_split
import livelossplot
import matplotlib.pyplot as plt

In [45]:
def DNN():
    # Defining our neural network model as a function

    model = Sequential()
    model.add(Dense(100, input_shape=(53,)))
    model.add(Dropout(0.001))

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.001))

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.001))

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.001))
    
    model.add(Dense(1))
    
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mae'])

    model.fit(X, Y, epochs=60, batch_size=8, verbose=0)

    y_pred = model.predict(new_X)
    y_test = new_Y
    std = (np.nanstd(abs(y_test-y_pred)))
    
    
    return m.mean_absolute_error(y_test, y_pred), m.mean_squared_error(y_test, y_pred), std

In [46]:
mae, mse, std = DNN()
print(mae, mse, std) 

313/313 [==============================] - 0s 1ms/step
7.693441971273353 93.0649108335136 15.282951374978241
